In [1]:
###########################################
# IMPORTS
###########################################
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools.constants import kB
from openmmtools.integrators import LangevinIntegrator

from simtk import unit, openmm

from perses.dispersed.utils import configure_platform
from perses.annihilation.lambda_protocol import LambdaProtocol, RelativeAlchemicalState, RESTState, RESTCapableRelativeAlchemicalState, RESTCapableLambdaProtocol
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
from perses.annihilation.relative import HybridTopologyFactory

import numpy as np
import itertools
import copy

import pickle
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
import mdtraj as md

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")

conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.4 jupyter extensions
DEBUG:matplotlib:matplotlib data path: /home/zhangi/miniconda3/envs/perses-rest-over-protocol/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/zhangi/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.4.3
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_

DEBUG:matplotlib:CACHEDIR=/home/zhangi/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/zhangi/.cache/matplotlib/fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


# Generate vanilla htf

In [3]:
# Create a htf
atp, system_generator = generate_atp(phase = 'solvent')
topology_proposal, new_positions, _, _ = generate_dipeptide_top_pos_sys(atp.topology, 
                               'THR', 
                               atp.system, 
                               atp.positions, 
                               system_generator,
                               flatten_torsions=True,
                               flatten_exceptions=True,
                               validate_endstate_energy=False)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:local_atom_map: {6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 14, 12: 15}
INFO:proposal_generator:the mapped atom names are: [('N', 'N'), ('H', 'H'), ('CA', 'CA'), ('HA', 'HA'), ('CB', 'CB'), ('C', 'C'), ('O', 'O')]


making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [14, 18, 15, 13, 16, 17, 19]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching functio

generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


...

conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 16.81406823661637


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 8.816525446865816
INFO:geometry:final reduced energy 25.63059342413189
INFO:geometry:sum of energies: 25.63059368348219
INFO:geometry:magnitude of difference in the energies: 2.593502959058469e-07
INFO:geometry:Final logp_proposal: -26900.657005873363


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.00017810081275281765), ('CustomTorsionForce', 0.004336815681512571), ('CustomBondForce', 8.812010530371548)]


In [5]:
flatten_exceptions = True
flatten_torsions = True
htf = HybridTopologyFactory(topology_proposal=topology_proposal,
                     current_positions=atp.positions,
                     new_positions=new_positions,
                     flatten_torsions=flatten_torsions,
                     interpolate_old_and_new_14s=flatten_exceptions
                    )

INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quant

In [6]:
with open("atp_solvent_vanilla.pickle", "wb") as f:
    pickle.dump(htf, f)

# Run vanilla MD on ala dipeptide at lambda = 1.0

In [7]:
with open("atp_solvent_vanilla.pickle", "rb") as f:
    htf = pickle.load(f)
    hybrid_system = htf.hybrid_system
    hybrid_positions = htf.hybrid_positions
    box_vectors = hybrid_system.getDefaultPeriodicBoxVectors()



In [11]:
# Create thermodynamic state
lambda_protocol = LambdaProtocol()
lambda_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_alchemical_state.set_alchemical_parameters(1, lambda_protocol)
compound_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(hybrid_system, temperature=temperature), composable_states=[lambda_alchemical_state])


In [12]:
# Get energy
integrator = LangevinIntegrator(temperature=300 * unit.kelvin, 
                                    collision_rate=1 / unit.picoseconds, 
                                    timestep=4 * unit.femtoseconds)
context = compound_thermodynamic_state.create_context(integrator)
context.setPositions(hybrid_positions)
context.setPeriodicBoxVectors(*box_vectors)
context.setVelocitiesToTemperature(300 * unit.kelvin)


In [13]:
for k, v in context.getParameters().items():
    print(k, v)

MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_angles 1.0
lambda_bonds 1.0
lambda_electrostatics_core 1.0
lambda_electrostatics_delete 1.0
lambda_electrostatics_insert 1.0
lambda_sterics_core 1.0
lambda_sterics_delete 1.0
lambda_sterics_insert 1.0
lambda_torsions 1.0
softcore_alpha 0.85


In [14]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)


In [15]:
energies, positions_old, positions_new = [], [], []

# for _ in tqdm_notebook(range(1000)):
#     integrator.step(250) 

for step in tqdm_notebook(range(250000)):
    integrator.step(1) 
    
#     if step > 1000:
    
#         # Get energy and positions
#         state = context.getState(getEnergy=True, getPositions=True)
#         energies.append(state.getPotentialEnergy().value_in_unit_system(unit.md_unit_system))
#         pos = state.getPositions(asNumpy=True)
#         old_pos = np.asarray(htf.old_positions(pos))
#         new_pos = np.asarray(htf.new_positions(pos))
#         positions_old.append(old_pos)
#         positions_new.append(new_pos)


/scratch/lsftmp/4086039.tmpdir/ipykernel_42821/1747201427.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm_notebook(range(250000)):


  0%|          | 0/250000 [00:00<?, ?it/s]

OpenMMException: Particle coordinate is nan

In [18]:
htf._atom_classes['core_atoms']


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19, 20, 21}

In [19]:
htf._atom_classes['unique_new_atoms']

{1548, 1549, 1550, 1551, 1552, 1553, 1554}

In [20]:
htf._atom_classes['unique_new_atoms']

{1548, 1549, 1550, 1551, 1552, 1553, 1554}

In [23]:
for i in range(hybrid_system.getNumParticles()):
    print(i, hybrid_system.getParticleMass(i))

0 4.0 Da
1 3.0340000000000007 Da
2 4.0 Da
3 4.0 Da
4 12.01 Da
5 16.0 Da
6 11.018 Da
7 4.0 Da
8 9.018 Da
9 4.0 Da
10 3.0340000000000007 Da
11 4.0 Da
12 4.0 Da
13 4.0 Da
14 12.01 Da
15 16.0 Da
16 11.018 Da
17 4.0 Da
18 3.0340000000000007 Da
19 4.0 Da
20 4.0 Da
21 4.0 Da
22 15.99943 Da
23 1.007947 Da
24 1.007947 Da
25 15.99943 Da
26 1.007947 Da
27 1.007947 Da
28 15.99943 Da
29 1.007947 Da
30 1.007947 Da
31 15.99943 Da
32 1.007947 Da
33 1.007947 Da
34 15.99943 Da
35 1.007947 Da
36 1.007947 Da
37 15.99943 Da
38 1.007947 Da
39 1.007947 Da
40 15.99943 Da
41 1.007947 Da
42 1.007947 Da
43 15.99943 Da
44 1.007947 Da
45 1.007947 Da
46 15.99943 Da
47 1.007947 Da
48 1.007947 Da
49 15.99943 Da
50 1.007947 Da
51 1.007947 Da
52 15.99943 Da
53 1.007947 Da
54 1.007947 Da
55 15.99943 Da
56 1.007947 Da
57 1.007947 Da
58 15.99943 Da
59 1.007947 Da
60 1.007947 Da
61 15.99943 Da
62 1.007947 Da
63 1.007947 Da
64 15.99943 Da
65 1.007947 Da
66 1.007947 Da
67 15.99943 Da
68 1.007947 Da
69 1.007947 Da
70 15.99943

1510 15.99943 Da
1511 1.007947 Da
1512 1.007947 Da
1513 15.99943 Da
1514 1.007947 Da
1515 1.007947 Da
1516 15.99943 Da
1517 1.007947 Da
1518 1.007947 Da
1519 15.99943 Da
1520 1.007947 Da
1521 1.007947 Da
1522 15.99943 Da
1523 1.007947 Da
1524 1.007947 Da
1525 15.99943 Da
1526 1.007947 Da
1527 1.007947 Da
1528 15.99943 Da
1529 1.007947 Da
1530 1.007947 Da
1531 15.99943 Da
1532 1.007947 Da
1533 1.007947 Da
1534 15.99943 Da
1535 1.007947 Da
1536 1.007947 Da
1537 15.99943 Da
1538 1.007947 Da
1539 1.007947 Da
1540 15.99943 Da
1541 1.007947 Da
1542 1.007947 Da
1543 15.99943 Da
1544 1.007947 Da
1545 1.007947 Da
1546 22.99 Da
1547 35.45 Da
1548 4.0 Da
1549 3.0340000000000007 Da
1550 4.0 Da
1551 4.0 Da
1552 4.0 Da
1553 13.008 Da
1554 4.0 Da


# Run vanilla MD on ala dipeptide at lambda = 0.0

In [7]:
with open("atp_solvent_vanilla.pickle", "rb") as f:
    htf = pickle.load(f)
    hybrid_system = htf.hybrid_system
    hybrid_positions = htf.hybrid_positions
    box_vectors = hybrid_system.getDefaultPeriodicBoxVectors()



In [8]:
# Create thermodynamic state
lambda_protocol = LambdaProtocol()
lambda_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_alchemical_state.set_alchemical_parameters(0, lambda_protocol)
compound_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(hybrid_system, temperature=temperature), composable_states=[lambda_alchemical_state])


In [9]:
# Get energy
integrator = LangevinIntegrator(temperature=300 * unit.kelvin, 
                                    collision_rate=1 / unit.picoseconds, 
                                    timestep=4 * unit.femtoseconds)
context = compound_thermodynamic_state.create_context(integrator)
context.setPositions(hybrid_positions)
context.setPeriodicBoxVectors(*box_vectors)
context.setVelocitiesToTemperature(300 * unit.kelvin)


In [10]:
for k, v in context.getParameters().items():
    print(k, v)

MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_angles 0.0
lambda_bonds 0.0
lambda_electrostatics_core 0.0
lambda_electrostatics_delete 0.0
lambda_electrostatics_insert 0.0
lambda_sterics_core 0.0
lambda_sterics_delete 0.0
lambda_sterics_insert 0.0
lambda_torsions 0.0
softcore_alpha 0.85


In [11]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)


In [12]:
energies, positions_old, positions_new = [], [], []

# for _ in tqdm_notebook(range(1000)):
#     integrator.step(250) 

for step in tqdm_notebook(range(250000)):
    integrator.step(1) 
    
#     if step > 1000:
    
#         # Get energy and positions
#         state = context.getState(getEnergy=True, getPositions=True)
#         energies.append(state.getPotentialEnergy().value_in_unit_system(unit.md_unit_system))
#         pos = state.getPositions(asNumpy=True)
#         old_pos = np.asarray(htf.old_positions(pos))
#         new_pos = np.asarray(htf.new_positions(pos))
#         positions_old.append(old_pos)
#         positions_new.append(new_pos)


/scratch/lsftmp/4086171.tmpdir/ipykernel_1276/1747201427.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step in tqdm_notebook(range(250000)):


  0%|          | 0/250000 [00:00<?, ?it/s]

In [13]:
htf._atom_classes['core_atoms']


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19, 20, 21}

In [14]:
htf._atom_classes['unique_new_atoms']

{1548, 1549, 1550, 1551, 1552, 1553, 1554}

In [15]:
htf._atom_classes['unique_new_atoms']

{1548, 1549, 1550, 1551, 1552, 1553, 1554}

In [16]:
for i in range(hybrid_system.getNumParticles()):
    print(i, hybrid_system.getParticleMass(i))

0 4.0 Da
1 3.0340000000000007 Da
2 4.0 Da
3 4.0 Da
4 12.01 Da
5 16.0 Da
6 11.018 Da
7 4.0 Da
8 9.018 Da
9 4.0 Da
10 3.0340000000000007 Da
11 4.0 Da
12 4.0 Da
13 4.0 Da
14 12.01 Da
15 16.0 Da
16 11.018 Da
17 4.0 Da
18 3.0340000000000007 Da
19 4.0 Da
20 4.0 Da
21 4.0 Da
22 15.99943 Da
23 1.007947 Da
24 1.007947 Da
25 15.99943 Da
26 1.007947 Da
27 1.007947 Da
28 15.99943 Da
29 1.007947 Da
30 1.007947 Da
31 15.99943 Da
32 1.007947 Da
33 1.007947 Da
34 15.99943 Da
35 1.007947 Da
36 1.007947 Da
37 15.99943 Da
38 1.007947 Da
39 1.007947 Da
40 15.99943 Da
41 1.007947 Da
42 1.007947 Da
43 15.99943 Da
44 1.007947 Da
45 1.007947 Da
46 15.99943 Da
47 1.007947 Da
48 1.007947 Da
49 15.99943 Da
50 1.007947 Da
51 1.007947 Da
52 15.99943 Da
53 1.007947 Da
54 1.007947 Da
55 15.99943 Da
56 1.007947 Da
57 1.007947 Da
58 15.99943 Da
59 1.007947 Da
60 1.007947 Da
61 15.99943 Da
62 1.007947 Da
63 1.007947 Da
64 15.99943 Da
65 1.007947 Da
66 1.007947 Da
67 15.99943 Da
68 1.007947 Da
69 1.007947 Da
70 15.99943

1131 1.007947 Da
1132 15.99943 Da
1133 1.007947 Da
1134 1.007947 Da
1135 15.99943 Da
1136 1.007947 Da
1137 1.007947 Da
1138 15.99943 Da
1139 1.007947 Da
1140 1.007947 Da
1141 15.99943 Da
1142 1.007947 Da
1143 1.007947 Da
1144 15.99943 Da
1145 1.007947 Da
1146 1.007947 Da
1147 15.99943 Da
1148 1.007947 Da
1149 1.007947 Da
1150 15.99943 Da
1151 1.007947 Da
1152 1.007947 Da
1153 15.99943 Da
1154 1.007947 Da
1155 1.007947 Da
1156 15.99943 Da
1157 1.007947 Da
1158 1.007947 Da
1159 15.99943 Da
1160 1.007947 Da
1161 1.007947 Da
1162 15.99943 Da
1163 1.007947 Da
1164 1.007947 Da
1165 15.99943 Da
1166 1.007947 Da
1167 1.007947 Da
1168 15.99943 Da
1169 1.007947 Da
1170 1.007947 Da
1171 15.99943 Da
1172 1.007947 Da
1173 1.007947 Da
1174 15.99943 Da
1175 1.007947 Da
1176 1.007947 Da
1177 15.99943 Da
1178 1.007947 Da
1179 1.007947 Da
1180 15.99943 Da
1181 1.007947 Da
1182 1.007947 Da
1183 15.99943 Da
1184 1.007947 Da
1185 1.007947 Da
1186 15.99943 Da
1187 1.007947 Da
1188 1.007947 Da
1189 15.99943 